### **Introduction**

In this project, I utilized Google’s Gemini model to develop an intelligent system for financial document analysis and personalized loan advisory, addressing a pressing issue in Kenya’s financial landscape as reported by a kenyan bank analysis report,I know this is not an issue facing this country only its world wide: the **rising rate of non-performing loans**. With more individuals and businesses struggling to repay loans—partly due to economic uncertainty and poor financial planning—there is an urgent need for smarter, data-driven tools that can both inform lending decisions and guide borrowers toward better financial health.

To begin, I took advantage of **Gemini’s large context window** to analyze the Kenya Financial Performance Document 2023, published by the Kenya Bureau of Standards (KEBS). This detailed national report contains extensive economic data, including trends that influence lending performance, credit access, and default risk. Processing such a large document in its entirety allowed the model to extract contextually relevant insights—such as sector-specific risks and macroeconomic trends—that are crucial for financial institutions trying to assess lending environments more accurately.

To make the system responsive to practical loan advisory needs, I applied **few-shot prompting**. By showing the model a handful of curated examples—such as how to evaluate a borrower’s spending behavior, savings discipline, or debt-to-income ratio:
—I was able to shape its outputs to give advice that aligns with real-world lending criteria. This approach helps borrowers receive tailored financial feedback, improving their preparedness and reducing the chances of default.

Furthermore, I enhanced user interaction by integrating **function calling**, enabling the model to work with real-world financial data from my M-Pesa statement. For context, M-Pesa is a widely used mobile money service in Kenya that allows people to send money, pay bills, and make purchases directly from their phones. I downloaded my M-Pesa statement, converted it into a CSV, and then imported it into a SQL table. This allowed the model to run natural language queries like “How much did I spend on recurring expenses last month?” or “What is my average transaction amount?”, turning complex financial behavior into digestible insights.

By combining these techniques—large-context analysis, few-shot learning, and functional calling —this project contributes a practical solution to the growing challenge of non performning loans. It empowers individuals with AI-driven financial literacy while giving lenders smarter tools to assess creditworthiness and manage risk more proactively.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/economic/Facts-Figures-2024.pdf
/kaggle/input/statements/ml_ready_features.csv
/kaggle/input/statements/merged_mpesa.csv


In [2]:
!pip install -qqqU PyPDF2
!pip install -qqqU edge-tts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
import google.generativeai as genai
from google.genai import types
import sqlite3
import PyPDF2
from IPython.display import HTML, Markdown, display

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("gemini_key")
genai.configure(api_key=secret_value)

In [5]:
generation_config = {
  "temperature": 0.3,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

## Gemini’s large context window to analyze the Kenya Financial Performance Document 2023,

In [6]:
pdf_file='/kaggle/input/economic/Facts-Figures-2024.pdf'

In [7]:
my_file = genai.upload_file(pdf_file)
file_name = my_file.name
myfile = genai.get_file(file_name)

print(myfile)

genai.File({
    'name': 'files/ccshom5lw93f',
    'display_name': 'Facts-Figures-2024.pdf',
    'mime_type': 'application/pdf',
    'sha256_hash': 'MmJhYjBjODdhNzBiM2U2YWY1MWNiNDU1YzAzYWNmOTU3YTk2Mjc1NWE4MzliM2NhN2I5NjJjMmRmNjFkY2JmYw==',
    'size_bytes': '8257036',
    'state': 'ACTIVE',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/ccshom5lw93f',
    'create_time': '2025-04-08T13:21:08.947744Z',
    'expiration_time': '2025-04-10T13:21:08.893386321Z',
    'update_time': '2025-04-08T13:21:08.947744Z'})


> testing the model on the pdf file

In [8]:
result = model.generate_content(
    [myfile, "\n\n", "what is the paper provided about?"]
)
Markdown(result.text)

The document is the Kenya Facts and Figures 2024 publication from the Kenya National Bureau of Statistics (KNBS).  It provides a summary of selected economic and social indicators for the period 2019 to 2023, presented in charts and tables.  The publication covers a wide range of topics including:

* **National Accounts:** GDP, income, savings, and economic activity by sector.
* **Employment and Earnings:** Wage employment, wage payments, and inflation.
* **Monetary and Financial Statistics:** Money supply, interest rates, and financial sector data.
* **Government Finance:** Revenue, expenditure, and public debt.
* **International Trade:** Exports, imports, and balance of trade.
* **Agriculture Sector:** Production of key crops.
* **Energy Sector:** Electricity generation and consumption, petroleum consumption.
* **Manufacturing Sector:** Manufacturing output.
* **Building and Construction Sector:** Road infrastructure.
* **Tourism Sector:** Visitor arrivals.
* **Transport Sector:** Passenger and cargo traffic.
* **Communication Sector:** Telephone and internet subscriptions.
* **Education Sector:** Student enrollment and budgets.
* **Health Sector:** Healthcare personnel and facilities.
* **Governance Sector:** Identity card applications, passports issued, and crime statistics.
* **Social and Economic Inclusion:** Beneficiaries of affirmative action funds and social protection programs.


Essentially, it's a statistical overview of key aspects of Kenya's economy and society during the specified period.  It's meant to be a concise reference, with more detailed information available in the full Economic Survey 2024 Report.


## few-shot prompting

In [9]:
few_shot_prompt="""
The provided document is a statistical report on Kenya, not a financial guide.use it when neccesary to provide tailored 
advice,for a particular request the advice should look like this :
With a monthly income of KES 20,000 and a 12-month repayment period, it's crucial
to apply for a loan amount that won't strain your finances. A general rule of thumb is
to keep your loan repayments at no more than 30-40% of your monthly income. Let's
analyze:
Maximum Recommended Loan:
 Using the 40% rule, your maximum monthly repayment should be KES 8,000 (KES
20,000
 0.40). To determine the maximum loan amount, you'll need to know the interest rate
offered by the lender. Let's assume an interest rate of 15% per annum (this is a
common rate, but you should check with lenders for their current rates). You can use
a loan calculator (many are available online) to determine the maximum loan amount
that results in a monthly repayment of approximately KES 8,000 over 12 months. The
actual amount will depend on the specific interest rate and any fees.
Conservative Approach:
 A more conservative approach, especially given the inherent risks in business,
would be to limit your monthly repayment to 30% of your income (KES 6,000). This
leaves more room for unexpected expenses and ensures you can comfortably meet
your obligations. Again, use a loan calculator to determine the corresponding loan
amount.
Investment Recommendations for Your Teaching Business:
Since you're a teacher with an established business, the loan should be used to
enhance your teaching capabilities and potentially increase your income. Here are
some investment options:
Improved Teaching Resources:
 Invest in high-quality teaching materials, technology (like a projector, interactive
whiteboard, or educational software), or a comfortable and well-equipped learning
space. This will improve the quality of your teaching and attract more students.
Marketing and Advertising:
 Allocate funds to promote your teaching services. This could involve creating a
professional website, running targeted online ads, distributing flyers in your
community, or networking with schools and other institutions.
Professional Development:
 Use some of the loan to attend workshops, conferences, or online courses to
enhance your teaching skills and knowledge in your specific subject area. This will
make you more competitive and attractive to potential students.
Expansion of Services:
 If feasible, consider expanding your services. This could involve offering tutoring
services, online courses, or workshops in addition to your regular teaching.
Important Considerations:
Interest Rates:
 Shop around and compare loan offers from different lenders in Kenya to find the
best interest rate.
Loan Fees:
 Be aware of any associated fees (processing fees, late payment penalties, etc.) and
factor them into your calculations.
Repayment Plan:
 Ensure you have a realistic repayment plan that aligns with your income and cash
flow.
Emergency Fund:
 Before taking out a loan, ensure you have an emergency fund to cover unexpected
expenses. This will protect you if your business experiences a downturn.
Before applying for any loan, create a detailed business plan outlining how you will
use the loan and how it will generate a return on investment. This will help you secure
the loan and ensure its successful utilization. """

borrowers_request = """what amount of loan do you recomend for a person having a cattle brokerage bussiness with a monthly income 
of ksh.60000 and on an established bussiness"""


result = model.generate_content(
    [myfile,few_shot_prompt, "\n\n", borrowers_request]
)
Markdown(result.text)

With a monthly income of KES 60,000, you have more flexibility in loan amounts compared to someone with a KES 20,000 income.  However, responsible borrowing remains crucial.

**Recommended Loan Amount Range:**

Using the 30-40% rule of thumb for loan repayments relative to monthly income, your maximum monthly repayment should be between KES 18,000 and KES 24,000.  The exact loan amount you can afford will depend on the interest rate and loan term.  Use an online loan calculator to determine the maximum loan amount given different interest rates and repayment periods.

**Investment Recommendations for Your Cattle Brokerage Business:**

Since you have an established cattle brokerage business, a loan should be used to expand your operations and potentially increase your income. Here are some investment options:

* **Improved Transportation:** Invest in reliable transportation (a vehicle or upgrading your current one) to efficiently move cattle and reduce transportation costs. This will allow you to handle more transactions and potentially increase your profit margins.

* **Technology and Communication:** Invest in communication tools (a better phone, reliable internet access) and potentially software to manage your cattle listings, track transactions, and communicate with buyers and sellers more efficiently.

* **Networking and Market Expansion:** Use some of the loan to attend livestock shows, trade fairs, or agricultural events to network with potential clients and expand your market reach.

* **Improved Record Keeping:** Invest in better record-keeping systems (software or hiring an assistant) to track your transactions, inventory, and client information. This will improve your business's efficiency and professionalism.

* **Training and Skill Enhancement:** Consider investing in training courses or workshops to improve your knowledge of cattle breeds, market trends, and negotiation skills.

**Important Considerations:**

* **Interest Rates:**  Compare loan offers from different lenders in Kenya to secure the lowest interest rate possible.

* **Loan Fees:** Factor in any associated fees (processing fees, early repayment penalties, etc.) when calculating your affordability.

* **Repayment Plan:** Create a realistic repayment plan that aligns with your cash flow.  Consider a shorter repayment period to minimize interest paid.

* **Emergency Fund:** Maintain a healthy emergency fund to cover unexpected expenses.

* **Business Plan:** Before applying, create a detailed business plan outlining how you'll use the loan and how it will generate a return on investment.  This will strengthen your loan application.


Remember, the specific loan amount you should seek depends on your individual circumstances and risk tolerance.  The suggestions above provide a framework for making an informed decision.  Always consult with a financial advisor for personalized guidance.


## function calling

In [10]:
%load_ext sql
%sql sqlite:///pandas_to_sql.db

In [11]:
df=pd.read_csv("/kaggle/input/statements/merged_mpesa.csv")
df1=pd.read_csv("/kaggle/input/statements/ml_ready_features.csv")

In [12]:
from io import StringIO

# Convert DataFrame to SQLite
db_file = "pandas_to_sql.db"
conn = sqlite3.connect(db_file)
df.to_sql("transuction", conn, if_exists="replace", index=False)
df1.to_sql("summary", conn, if_exists="replace", index=False)

4

In [13]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables()')

    cursor = conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()

 - DB CALL: list_tables()


['transuction', 'summary']

In [14]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    print(f' - DB CALL: execute_query({sql})')

    cursor = conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


In [15]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(f' - DB CALL: describe_table({table_name})')

    cursor = conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]


describe_table("transuction")

 - DB CALL: describe_table(transuction)


[('Receipt No.', 'TEXT'),
 ('Completion Time', 'TEXT'),
 ('Details', 'TEXT'),
 ('Transaction Status', 'TEXT'),
 ('Paid In', 'TEXT'),
 ('Withdrawn', 'TEXT'),
 ('Balance', 'TEXT'),
 ('Unnamed: 2', 'TEXT'),
 ('Unnamed: 3', 'TEXT'),
 ('Unnamed: 4', 'TEXT')]

In [16]:
describe_table("summary")

 - DB CALL: describe_table(summary)


[('Month', 'TEXT'),
 ('Paid In', 'REAL'),
 ('Withdrawn', 'REAL'),
 ('Net Amount', 'REAL'),
 ('Transaction Count', 'INTEGER'),
 ('Bill Payment', 'INTEGER'),
 ('Gambling', 'INTEGER'),
 ('Income', 'INTEGER'),
 ('Loan Related', 'INTEGER'),
 ('Other', 'INTEGER'),
 ('Peer Transfer', 'INTEGER'),
 ('Utilities', 'INTEGER'),
 ('Loan Expense Ratio', 'REAL'),
 ('Bill Expense Ratio', 'REAL')]

In [17]:
from google import genai
from kaggle_secrets import UserSecretsClient


GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)
# These are the Python functions defined above.
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot that can interact with an SQL database
for a computer store. You will take the users questions and turn them into SQL
queries using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use list_tables to see what tables are present, describe_table to understand the
schema, and execute_query to issue an SQL SELECT query."""

client = genai.Client(api_key=GOOGLE_API_KEY)

# Start a chat with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)

In [18]:
resp = chat.send_message("how many transuctions involved moureen on the Details column of transuction table?")
print(f"\n{resp.text}")

 - DB CALL: execute_query(SELECT count(*) FROM transuction WHERE Details LIKE '%moureen%';)

There are 48 transactions that involved "moureen" in the Details column of the transuction table.



In [19]:
resp = chat.send_message("can i see them")
Markdown(f"\n{resp.text}")

 - DB CALL: execute_query(SELECT * FROM transuction WHERE Details LIKE '%moureen%';)



Here are the 48 transactions that involved "Moureen Wanga" in the details:

TA32TLYC5G, 2025-01-03 15:43:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.0, None, 100.0, None, None, None
TA34TM3HGW, 2025-01-03 15:44:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.0, None, 100.0, None, None, None
TAE27N8QI0, 2025-01-14 20:42:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.00, None, 100.00, None, None, None
TAE47NZ0FY, 2025-01-14 20:46:00, Funds received from - 07******134 Moureen Wanga, Completed, 40.00, None, 110.11, None, None, None
TAF0AKZBX6, 2025-01-15 17:01:00, Funds received from - 07******134 Moureen Wanga, Completed, 2,000.00, None, 2,000.00, None, None, None
TAF8ALMOD8, 2025-01-15 17:05:00, Funds received from - 07******134 Moureen Wanga, Completed, 40.00, None, 1,969.91, None, None, None
TAG9FQX7QH, 2025-01-16 19:50:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.00, None, 1,209.91, None, None, None
TAG6FRID5S, 2025-01-16 19:52:00, Funds received from - 07******134 Moureen Wanga, Completed, 40.00, None, 1,249.91, None, None, None
TBN76LM6AH, 2025-02-23 17:34:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.00, None, 1,289.67, None, None, None
TBN56LZ37D, 2025-02-23 17:36:00, Funds received from - 07******134 Moureen Wanga, Completed, 50.00, None, 1,339.67, None, None, None
TBO5C9P2DJ, 2025-02-24 20:59:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.00, None, 1,332.67, None, None, None
TBO5CA6IV3, 2025-02-24 21:01:00, Funds received from - 07******134 Moureen Wanga, Completed, 50.00, None, 1,382.67, None, None, None
TBR5OSAI91, 2025-02-27 18:04:00, Customer Transfer to - 07******134 Moureen Wanga, Completed, None, -500.0, 4,967.67, None, None, None
TC214J0S4B, 2025-03-02 19:28:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.0, None, 1,193.67, None, None, None
TC224KEJ5A, 2025-03-02 19:33:00, Funds received from - 07******134 Moureen Wanga, Completed, 50.0, None, 1,243.67, None, None, None
TC329OUB30, 2025-03-03 20:24:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.0, None, 948.67, None, None, None
TC319P3K77, 2025-03-03 20:25:00, Funds received from - 07******134 Moureen Wanga, Completed, 50.0, None, 998.67, None, None, None
TC60NME4IE, 2025-03-06 19:41:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.0, None, 843.67, None, None, None
TC62NNASOY, 2025-03-06 19:44:00, Funds received from - 07******134 Moureen Wanga, Completed, 50.0, None, 893.67, None, None, None
TC76SQN50O, 2025-03-07 20:33:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.00, None, 863.67, None, None, None
TC78SR7QIC, 2025-03-07 20:35:00, Funds received from - 07******134 Moureen Wanga, Completed, 50.00, None, 913.67, None, None, None
TC83WVQFGD, 2025-03-08 19:07:00, Funds received from - 07******134 Moureen Wanga, Completed, 150.00, None, 913.67, None, None, None
TC80XQ43KE, 2025-03-08 21:16:00, Funds received from - 07******134 Moureen Wanga, Completed, 3,000.00, None, 3,913.67, None, None, None
TC932IASAP, 2025-03-09 19:26:00, Funds received from - 07******134 Moureen Wanga, Completed, 150.00, None, 3,953.67, None, None, None
TCB6BR8Q22, 2025-03-11 20:15:00, Funds received from - 07******134 Moureen Wanga, Completed, 150.00, None, 1,354.67, None, None, None
TCC6G16DT6, 2025-03-12 19:32:00, Funds received from - 07******134 Moureen Wanga, Completed, 150.00, None, 1,384.67, None, None, None
TCE2MLUUFO, 2025-03-14 10:38:00, Funds received from - 07******134 Moureen Wanga, Completed, 150.00, None, 1,257.67, None, None, None
TCH342UM7D, 2025-03-17 20:57:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.00, None, 1,342.67, None, None, None
TCI68C1DO8, 2025-03-18 20:29:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.00, None, 1,302.67, None, None, None
TCI68CUUFG, 2025-03-18 20:33:00, Funds received from - 07******134 Moureen Wanga, Completed, 40.00, None, 1,342.67, None, None, None
TCJ3CCM46R, 2025-03-19 19:37:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.00, None, 1,222.67, None, None, None
TCJ5CCVH9T, 2025-03-19 19:38:00, Funds received from - 07******134 Moureen Wanga, Completed, 50.00, None, 1,272.67, None, None, None
TCK9G9EDNT, 2025-03-20 18:29:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.00, None, 7,085.67, None, None, None
TCO3Y771W7, 2025-03-24 19:28:00, Funds received from - 07******134 Moureen Wanga, Completed, 47.00, None, 47.00, None, None, None
TCP83QOK9Q, 2025-03-25 19:50:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.00, None, 141.65, None, None, None
TCP63R39FY, 2025-03-25 19:52:00, Funds received from - 07******134 Moureen Wanga, Completed, 50.00, None, 191.65, None, None, None
TCQ08957L0, 2025-03-26 20:03:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.00, None, 819.65, None, None, None
TCQ589E8MJ, 2025-03-26 20:04:00, Funds received from - 07******134 Moureen Wanga, Completed, 50.00, None, 869.65, None, None, None
TCU1OB0DO9, 2025-03-30 11:40:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.0, None, 922.65, None, None, None
TCU6OB9K3E, 2025-03-30 11:42:00, Funds received from - 07******134 Moureen Wanga, Completed, 50.0, None, 972.65, None, None, None
TCV3V1XBO3, 2025-03-31 19:40:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.0, None, 752.65, None, None, None
TCV9V2FLVT, 2025-03-31 19:42:00, Funds received from - 07******134 Moureen Wanga, Completed, 20.0, None, 772.65, None, None, None
TD15ZFVUAJ, 2025-04-01 18:57:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.0, None, 722.65, None, None, None
TD15ZG8PWJ, 2025-04-01 18:58:00, Funds received from - 07******134 Moureen Wanga, Completed, 50.0, None, 772.65, None, None, None
TD295PBX8R, 2025-04-02 19:53:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.0, None, 487.65, None, None, None
TD329WDUGC, 2025-04-03 18:22:00, Funds received from - 07******134 Moureen Wanga, Completed, 150.0, None, 437.65, None, None, None
TD43F5H5XN, 2025-04-04 19:10:00, Funds received from - 07******134 Moureen Wanga, Completed, 100.0, None, 447.65, None, None, None
TD48F5Z54Y, 2025-04-04 19:12:00, Funds received from - 07******134 Moureen Wanga, Completed, 50.0, None, 497.65, None, None, None

## conclution

Wow!this is great and huge thank you to the google team for the wonderfull course.